<a id='top'></a>

# Structural damping

Table of contents:

[Introduction](#Introduction)

[Dynamic equilibrium equation](#Dynamic-equilibrium-equation)

[Computational lab](#Computational-lab)

[Stiffness proportional damping](#Stiffness-proportional-damping)

[Mass proportional damping](#Mass-proportional-damping)

[Rayleigh damping](#Rayleigh-damping)

[Odds and ends](#Odds-and-ends)

References:

http://www.sciencedirect.com/science/article/pii/S0020768313003727

http://www.sandv.com/downloads/1306schw.pdf

http://engweb.swan.ac.uk/~adhikaris/fulltext/conference/ftc27.pdf

https://wiki.csiamerica.com/display/kb/Damping+coefficients

https://www.orcina.com/SoftwareProducts/OrcaFlex/Documentation/Help/Content/html/RayleighDamping,Guidance.htm

## Introduction

In structural dynamics, mass and stiffness can be computed directly from the geometric characteristics and material properties of a structure but damping can only be estimated based on the fact that structural dynamic responses are, well, damped. It is usually assumed that structural damping is viscous, in the absence of more accurate information, which fits nicely in the [dynamic equilibrium equation](#Dynamic-equilibrium-equation).

This python notebook will explore three types of viscous damping normally considered in the computation of dynamic responses:

1. [Stiffness proportional damping](#Stiffness-proportional-damping)
2. [Mass proportional damping](#Mass-proportional-damping)
3. [Rayleigh damping](#Rayleigh-damping)

When using this type of damping, the "damping" matrix is orthogonal to the modal matrix and, therefore, is diagonalizable. This constitutes a clear advantage when it comes to solving the dynamic equilibrium equation system for multi-degree-of-freedom systems.

[Back to top](#top)

## Dynamic equilibrium equation

In structural dynamics the second order differential dynamic equilibrium equation can be written in terms of generalized coordinates (d[isplacement]) and their first (v[elocity]) and second (a[cceleration]) time derivatives:

\begin{equation}
\mathbf{M} \times \mathbf{a(t)} + \mathbf{C} \times \mathbf{v(t)} + \mathbf{K} \times \mathbf{d(t)} = \mathbf{F(t)}
\end{equation}

where:

> $\mathbf{M}$ is the mass matrix  
> $\mathbf{C}$ is the damping matrix  
> $\mathbf{K}$ is the stiffness matrix  
> $\mathbf{a(t)}$ is the acceleration vector  
> $\mathbf{v(t)}$ is the velocity vector  
> $\mathbf{d(t)}$ is the displacement vector  
> $\mathbf{F(t)}$ is the force input vector  

All these matrices are of size $NDOF \times NDOF$, where $NDOF$ is the number of generalized degrees of freedom of the dynamic system. Carrying out the usual coordinate transformation from generalized coordinates to modal coordinates, $\mathbf{d(t)} = \mathbf{\Phi}^T \times \mathbf{q(t)}$, and pre-multiplying by the transpose of the modal matrix, $\mathbf{\Phi}^T$, one obtains the following:

\begin{equation}
\mathbf{\Phi}^T \times \mathbf{M} \times \mathbf{\Phi} \times \mathbf{\ddot q(t)} + \mathbf{\Phi}^T \times \mathbf{C} \times \mathbf{\Phi} \times \mathbf{\dot q(t)} + \mathbf{\Phi}^T \times \mathbf{K} \times \mathbf{\Phi} \times \mathbf{q(t)} = \mathbf{\Phi}^T \times \mathbf{F(t)}
\end{equation}

where:

> $\mathbf{\Phi}^T \times \mathbf{M} \times \mathbf{\Phi} = \mathbf{M_n}$ is a diagonal matrix (modal masses)  
> $\mathbf{\Phi}^T \times \mathbf{K} \times \mathbf{\Phi} = \mathbf{K_n}$ is a diagonal matrix (modal stiffness)  

The product $\mathbf{\Phi}^T \times \mathbf{C} \times \mathbf{\Phi}$ usually is not a diagonal matrix.
This equivalent equation system...

[Back to top](#top)

## Mass proportional damping

Mass proportional damping means that the damping coefficient is somehow a multiple of mass:

$$\mathbf{C} = \alpha \times \mathbf{M}$$

where $\alpha$ is the constant of mass proportionality. In these circunstances, the dynamic equilibrium equation can be written as:

\begin{equation}
\mathbf{M} \times \mathbf{a(t)} + \alpha \times \mathbf{M} \times \mathbf{v(t)} + \mathbf{K} \times \mathbf{d(t)} = \mathbf{F(t)}
\end{equation}

Proceeding as above, one obtains the independent modal equilibrium equations:

\begin{equation}
\mathbf{M_n} \times \mathbf{a_n(t)} + \alpha \times \mathbf{M_n} \times \mathbf{v_n(t)} + \mathbf{K_n} \times \mathbf{d_n(t)} = \mathbf{F_n(t)}
\end{equation}

The solution of this equation system is:

[Back to top](#top)

## Stiffness proportional damping

Stiffness proportional damping means that the damping coefficient is somehow a multiple of stiffness:

$$\left[ C \right] = \beta \times \left[ K \right]$$

where $\beta$ is the constant of stiffness proportionality. In these circunstances, the dynamic equilibrium equation can be written as:

\begin{equation}
\mathbf{M} \times \mathbf{a(t)} + \beta \times \mathbf{K} \times \mathbf{v(t)} + \mathbf{K} \times \mathbf{d(t)} = \mathbf{F(t)}
\end{equation}

or, after pre-multiplying by $\mathbf{K}^{-1}$,

\begin{equation}
\mathbf{K}^{-1} \times \mathbf{M} \times \mathbf{a(t)} + \beta \times \mathbf{v(t)} + \mathbf{d(t)} = \mathbf{K}^{-1} \times \mathbf{F(t)}
\end{equation}

The solution of this equation system is:


## Computational lab

In [1]:
import sys
import math
import numpy as np
import matplotlib as mpl
print('System: {}'.format(sys.version))
for package in (np, mpl):
    print('Package: {} {}'.format(package.__name__, package.__version__))

System: 3.5.2 |Anaconda custom (64-bit)| (default, Jul  5 2016, 11:41:13) [MSC v.1900 64 bit (AMD64)]
Package: numpy 1.11.2
Package: matplotlib 1.5.3


In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

[Back to top](#top)

## Mass proportional damping

Mass proportional damping means that the damping coefficient is somehow a multiple of mass:

$$\left[ C \right] = \alpha \times \left[ M \right]$$

where $\alpha$ is the constant of mass proportionality. In these circunstances, the dynamic equilibrium equation can be written as:

\begin{equation}
\mathbf{M} \times \mathbf{a(t)} + \alpha \times \mathbf{M} \times \mathbf{v(t)} + \mathbf{K} \times \mathbf{d(t)} = \mathbf{F(t)}
\end{equation}

or, after pre-multiplying by $\mathbf{M}^{-1}$,

\begin{equation}
\mathbf{a(t)} + \alpha \times \mathbf{v(t)} + \mathbf{M}^{-1} \times \mathbf{K} \times \mathbf{d(t)} = \mathbf{M}^{-1} \times \mathbf{F(t)}
\end{equation}

The solution of this equation system is:


[Back to top](#top)

## Stiffness proportional damping

Stiffness proportional damping means that the damping coefficient is somehow a multiple of stiffness:

$$\left[ C \right] = \beta \times \left[ K \right]$$

where $\beta$ is the constant of stiffness proportionality. In these circunstances, the dynamic equilibrium equation can be written as:

\begin{equation}
\mathbf{M} \times \mathbf{a(t)} + \beta \times \mathbf{K} \times \mathbf{v(t)} + \mathbf{K} \times \mathbf{d(t)} = \mathbf{F(t)}
\end{equation}

or, after pre-multiplying by $\mathbf{K}^{-1}$,

\begin{equation}
\mathbf{K}^{-1} \times \mathbf{M} \times \mathbf{a(t)} + \beta \times \mathbf{v(t)} + \mathbf{d(t)} = \mathbf{K}^{-1} \times \mathbf{F(t)}
\end{equation}

The solution of this equation system is:


[Back to top](#top)

## Rayleigh damping

Rayleigh damping means that the damping coefficient is a combination of the former, that is, it is a multiple of mass and stifnness:

$$\left[ C \right] = \alpha \times \left[ M \right] + \beta \times \left[ K \right]$$

where $\alpha$ and $\beta$ have the previous meanings.

[Back to top](#top)

## Odds and ends

This notebook was created by Paulo Xavier Candeias.

[Back to top](#top)